In [57]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

测试样例：生成torch.arange(1,7).resize_(2,3)

数据集准备:两句话"德语"转化为"英语"

In [58]:
# P: 长度填充符号，为的是使得每个batch中的语句长度一样
# S: decoder输入时第一个单词
# E: decoder输出时最后一个单词
sentences = [
        # 德语                      # 训练时decoder的输入       # decoder的输出
        ['ich mochte ein bier P',  'S i want a beer . ', 'i want a beer . E'],
        ['ich mochte ein cola P',  'S i want a coke . ', 'i want a coke . E']
]

# 对src语言和target语言进行分别编码
src_vocab = {'P' : 0, 'ich' : 1, 'mochte' : 2, 'ein' : 3, 'bier' : 4, 'cola' : 5}
tgt_vocab = {'P' : 0, 'i' : 1, 'want' : 2, 'a' : 3, 'beer' : 4, 'coke' : 5, '.' : 6, 'S' : 7, 'E' : 8}
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)
# target编码转化为语言(用于输出)
idx2word = {i: w for i, w in enumerate(tgt_vocab)}

# src_len = 5 # src语言最大输入长度(此例中是5)(enc_input最大长度)
# tgt_len = 6 # target语言最大输入长度(dec_input(=dec_output)长度)

# 将一句话转化为编码，并且返回对应的目标编码
def make_data(sentence):
    enc_input = [src_vocab[n] for n in sentence[0].split()]
    dec_input = [tgt_vocab[n] for n in sentence[1].split()]
    dec_output = [tgt_vocab[n] for n in sentence[2].split()]
    return torch.LongTensor(enc_input), torch.LongTensor(dec_input), torch.LongTensor(dec_output)

# 构建dataset重写dataset函数
class MyDataSet(Dataset):
    def __init__(self, sentences):
        super(MyDataSet, self).__init__()
        self.data = sentences
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        enc_input, dec_input, dec_output = make_data(self.data[idx])
        return enc_input, dec_input, dec_output


train_dataset = MyDataSet(sentences)
train_loader = DataLoader(train_dataset, batch_size = 2, shuffle = True)
# 测试样例
a = iter(train_loader)
a,b,c = a.next() # a = [batch_size, len(enc_input)] b =[batch_size, len(dec_input)] c = [batch_size, len(dec_output)]

参数定义
- d_model：Embedding_Size词向量的大小
- d_ff: FeedForward层隐藏神经元个数
- d_k : K向量维度   其中 Q 与 K 的维度必须相等，V 的维度没有限制
- d_v : V向量维度
- n_layers: Encoder和Decoder的层数
- n_heads：多头注意力机制的头个数

In [59]:
d_model = 512   # Embedding Size
d_ff = 2048     # FeedForward dimension
d_k = d_q = 64  # dimension of K(=Q)
d_v = 64        # dimension of V
n_layers = 6    # number of Encoder of Decoder Layer
n_heads = 8     # number of heads in Multi-Head Attention

## Positional Encoding(位置编码)
$PE(pos, 2i) = sin(pos/10000^{2i/d_{model}})$

$PE(pos, 2i+1) = cos(pos/10000^{2i/d_{model}})$

函数解释：
- nn.Dropout(p):以概率p将向量中某些位置元素归零
- torch.zeros():返回指定形状的全0张量
- tenor.unsqueeze(dim)：在指定位置生成一个维度，squeeze(dim)：在指定位置删除一个维度
- torch.arange(a, b, step=c, dtype=torch.float):在[a,b)以步长为c，生成一维张量
- tenor.transpose(dim1, dim2): 将tensor的两个维度进行转置(注意和view的差别)
- self.register_buffer('name', tensor):将tensor放入到模型缓冲区，使得其不会被optim.step更新，但是在载入的时候仍会被载入。

参数解释：
- max_len: 生成的位次最大长度，如果一句话的单词数超过max_len，则需要更改这个参数来即seq_len <= max_len 
- dropout: 会以概率dropout来遮盖返回张量。制造噪声。


In [60]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)    # [max_len, d_model]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # shape:[max_len] ->[max_len,1]
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * (-math.log(10000.0) / d_model)) #shape:[256]
        pe[:, 0::2] = torch.sin(position * div_term)   # 得到偶数位置
        pe[:, 1::2] = torch.cos(position * div_term)   # 得到奇数位置
        pe = pe.unsqueeze(1)            # [max_len, 1, d_model]
        self.register_buffer('pe', pe)  # 接下来的代码里可以使用self.pel来代替这个变量。

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        # pe[:seq_len, :]:[seq_len, 1, d_model] 此处仍用到了广播机制
        # 将pe[:seq_len, :]复制成[seq_len, batch_size, d_model]来进行相加
        x = x + self.pe[:x.size(0), :]  
        return self.dropout(x)

## Pad Mask
句子不够长，加了pad，因此需要对pad的元素进行mask，从而使得其在attention机制中占比为0，主要是在每一个句子都会产生一个相关度矩阵，表示不同的单词间通过K,Q，V计算出来的相关程度，但是我们需要把pad的关系全部变成0.函数返回的是一个元素为bool的张量。

函数解释：
- tensor.eq(a):判断tensor中每一个元素是否等于a，如果是则在对应位置返回T，否则返回F
- tensor.expend(a,b,c):函数可以将张量广播到新的形状，但是切记以下两点：1、只能对维度值为1的维度进行扩展，且扩展的Tensor不会分配新的内存，只是原来的基础上创建新的视图并返回；2、无需扩展的维度请保持维度值不变。

参数解释：
- seq_q: 产生Q矩阵的输入
- seq_k: 产生K矩阵的输入
- 这里之所以不一样是因为在Decoder中产生Q,K矩阵的来源是不同的（Decoder中的Q矩阵来自于Outputs，K,V矩阵是来自于Encoder的得到的结果）

注意：
- 此处我们mark只mark了列，换句话说PAD对其他字母还是具有注意力的，只不过其他字母对于mark没有了注意力
- 此处最会返回的结果类似如下[batch_size=3, len_q=2, len_k=6]
tensor([[[ F,  F,  F,  F,  T,  T],
         [ F,  F,  F,  F,  T,  T]],

        [[ F,  F,  F,  F,  F,  T],
         [ F,  F,  F,  F,  F,  T]],

        [[ F,  F,  F,  T,  T,  T],
         [ F,  F,  F,  T,  T,  T]]], dtype=torch.int32)

In [61]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, len_q]
    seq_k: [batch_size, len_k]
    return：[batch_size, len_q, len_k] 得到元素是bool的Tensor
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)          # [batch_size, 1, len_k], True is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k] 

## Subsequence Mask
只有 Decoder layer会用到(在第一个attention的时候)，主要作用是屏蔽未来时刻单词的信息。

函数解释：
- np.triu(np.array, k=0/1/-1)：将np.array返回成一个上三角矩阵，如果k=0，则对角线上有元素，k=1对角线上没有元素，k=-1，对角线下移一个对角线也有元素。

In [62]:
def get_attn_subsequence_mask(seq):
    '''
    seq: [batch_size, tgt_len]
    return：[batch_size, tgt_len, tgt_len] 得到元素是0，1的上三角阵，对角线数字是0
    '''
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)        # 生成上三角矩阵(对角线上有1)
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()    # 转化tensor格式为torch.uint8
    return subsequence_mask              # [batch_size, tgt_len, tgt_len]

## ScaledDotProductAttention
缩放注意力计算函数,通过K,Q计算出scores，然后将scores和V相乘，得到每个单词的context vector(也就是每个单词和其他的单词的相关程度)
![](缩放注意力计算函数.png)

- 第一步是将Q和K_T相乘,相乘之后得到的scores还不能立刻进行softmax(因为存在PAD也计算了)，需要和attn_mask相加，attn_mask是一个仅由 True 和 False 组成的tensor，并且一定会保证attn_mask和scores的维度四个值\[batch_size, n_heads, seq_len, seq_len]相同.
- mask完了之后，就可以对scores进行softmax了。然后再与V\[seq_len, d_v]相乘，得到context

函数解释：
- torch.matmul(a,b):把a和b做矩阵乘法，其中a,b可以是高维的,如果a，b是高维会将最高维提出来，将剩下的维度两个维度做矩阵乘法。
- tensor.masked_fill_(tensor1, value): tensor1是一个只有T,F的张量，函数将会把tensor对应tensor1为T的位置替换为value，相当于tensor1作为指标告诉函数，tensor的那些位置需要被替换

In [63]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_q]
        K: [batch_size, n_heads, len_k, d_k(=d_q)]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, len_q, len_k]  元素是bool的Tensor
        return: context(加权之后的v)，attn(单词间的相关关系矩阵)
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)  # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9)        # 将PAD的位置变成-1e9，在执行softmax的时候会变成0
        
        attn = nn.Softmax(dim=-1)(scores)           # 对最后一个维度(对每一行进行，把最后两个维度提出来看作矩阵的话)做softmax
                                                    # attn = [batch_size, n_heads, len_q, len_k],单词的权重
        context = torch.matmul(attn, V)             # context = [batch_size, n_heads, len_q, d_v]，最后得到的context值
        return context, attn

## MultiHeadAttention
多头attention模块，产生Wq,Wk,Wv并输出最后的

参数含义：
- input_Q：需要去乘以Wq得到Q矩阵的输入(在encoder中其实就是X矩阵),第三个维度可能与input_K的第三个维度是不同的因为在decoder的第二个attention中提供K,Q的是不同的数据
- input_K：需要去乘以Wq得到K矩阵的输入(在encoder中其实就是X矩阵)
- input_V：需要去乘以Wq得到V矩阵的输入(在encoder中其实就是X矩阵)
- attn_mask： 用于处理PAD使用的(在decoder中为了掩盖后续信息)

函数解释：
- tensor.repeat(a,b,c,d): 其与expend一样但是expend不会改变原来的内存，repeat会改变原来内存，将对应维度扩展，比如\[2,12,2].repeat(1,3,1)将第2维度广播3遍得到\[2,36,2]

In [64]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)  # 全连接层
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, len_q, len_k]   元素是bool的Tensor
        return: attn(单词间的相关关系矩阵)
        '''
        residual, batch_size = input_Q, input_Q.size(0)     # 将Q作为residual残差，与之后的output相加
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)    # attn_mask : [batch_size, n_heads, len_q, len_k],屏蔽PAD的注意力

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v],合并多头
        output = self.fc(context)           # 对合并后的多头进行全连接层的映射，得到输出[batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).cuda()(output + residual), attn    # 做残差以及层归一化，注意这里使用了cuda()

## FeedForward Layer
对多头出来的结果进行双层的前向传播，再进行残差合并以及层归一化

函数解释：
- nn.Sequential(fun1(), fun2()...): 类似于函数集合，将函数fun1，fun2...定义好放入，之后只需要调用这个一个集合函数即可以完成整个函数集的调用

In [65]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, len_q(seq_len), d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).cuda()(output + residual)     # [batch_size, len_q(seq_len), d_model]

## Encoder Layer
这是单层的encoder层,用于拼装以上组件

In [66]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()  # 之前定义好的多头注意力机制
        self.pos_ffn = PoswiseFeedForwardNet()     # 之前定义的FeedForward

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]  用于消除PAD的注意力，元素是bool的Tensor
        return：attn(单词间的相关关系矩阵)
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]单词间的相关关系矩阵
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # inputQ,inputK,inputV的是相同，因此在多头注意力函数中len_q=len_k=len_v=src_len
        enc_outputs = self.pos_ffn(enc_outputs)     # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

## Encoder
将encoder层组合起来最终形成Encoder

函数解释：
- nn.ModuleList(\[fun1,fun2])：和nn.Sequential类似，不过里面是通过列表把定义好的fun1...传进去，在有些时候比如这里使用的是多层的同样的layer会比较方便。但是需要通过for循环拉出来使用，不能像nn.Sequential一样直接传入参数。
- nn.Embedding(vocab_size, d_model): 定义词嵌入时vocab_size表示词典的大小，d_model表示输出多少维度的向量来表示一个符号

In [67]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)    # 对句子做embedding
        self.pos_emb = PositionalEncoding(d_model)              # 将词向量中嵌入位置向量
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len] 得到元素是bool的Tensor
        enc_self_attns = []     # 用于收集每一层encoder layer的attn
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

## Decoder Layer
这是Decoder层，也是之前的组件进行的拼装。
- 会调用两次 MultiHeadAttention，第一次是计算Decoder_Input的 self-attention，得到输出dec_outputs。
- 然后将 dec_outputs 作为生成 Q 的输入，enc_outputs作为生成K和V的输入，再调用一次MultiHeadAttention，得到的是Encoder和Decoder Layer 之间的 context vector。最后将 dec_outputs 做一次维度变换，然后返回.

参数含义：
- dec_inputs: 由编码后得到的目标句子的词向量。也就是decoder的第一个attention的输入
- en_outputs: 经历过encoder得到的最终结果，用于在decoder的第二个attention提供生成K和V的输入
- dec_self_attn_mask : 对第一个attention使得dec_inputs没有办法看到后面的信息，只能看到当前即以前的信息。
- dec_enc_attn_mask : 对第二个attention使得屏蔽PAD的相关关系。

In [68]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        # dec_outputs: [batch_size, tgt_len, d_model], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn, dec_enc_attn

## Decoder
对原始输入进行embeding,并组装Decoder Layer形成输出，注意到这里的参数虽然也叫dec_inputs，但是和Decoder Layer的参数不是一个意思。

参数解释：
- dec_inputs: 对目标句子编码后的向量例如\[S我爱你PP]——>\[8,1,2,3,0,0]
- enc_inputs: 对原始句子编码后的向量例如\[I love you P]——>\[1,4,9,0]，输入这个信息是因为在DecoderLayer的第二个attention我们从encoder接收到了传出来的每个字的向量，但是里面的产生K的向量包含有PAD，因此我们在做第二个attention需要把这部分信息PAD掉，因此需要得知些PAD的位置。
- enc_outputs : 通过Encoder编码之后的数据，在DecoderLayer会用到，因此需要输入

函数解释：
- torch.gt(a, value)：将a的各位置与value比较，如果>value,则为该位置为1，否则为0

In [69]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        '''
        dec_inputs: [batch_size, tgt_len] 目标语言的编码之后的句子
        enc_intpus: [batch_size, src_len] 原语言的编码之后的句子
        enc_outputs: [batch_size, src_len, d_model] encoder得到的输出
        '''
        dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).cuda() # [batch_size, tgt_len, d_model]
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda()    # [batch_size, tgt_len, tgt_len]
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        # 不仅要把 "pad " mask 掉，还要 mask 未来时刻的信息，因此就有了下面这一行代码
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0).cuda() # [batch_size, tgt_len, tgt_len] 元素是0，1，1的位置就是不需要计算相关度的位置。

        
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs) # [batch_size, tgt_len, src_len]

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)    # 收集每一层decoder中第一个attention的得到的相关矩阵
            dec_enc_attns.append(dec_enc_attn)      # 收集每一层decoder中第二个attention的得到的相关矩阵  
        return dec_outputs, dec_self_attns, dec_enc_attns

## Transformer
主要是用来拼装Encoder和Decoder


In [70]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False).cuda()     # 最后需要对单词做分类，所以需要一个映射到目标字典这么长
    def forward(self, enc_inputs, dec_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        return：[batch_size * tgt_len, tgt_vocab_size]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        # dec_outpus: [batch_size, tgt_len, d_model],
        # dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len]
        # dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns

## 模型 & 损失函数 & 优化器
这里的损失函数里面我设置了一个参数 ignore_index=0，因为 "pad" 这个单词的索引为 0，这样设置以后，就不会计算 "pad" 的损失（因为本来 "pad" 也没有意义，不需要计算）

In [71]:
model = Transformer().cuda()
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)

## 训练

In [74]:
for epoch in range(20):
    for enc_inputs, dec_inputs, dec_outputs in train_loader:
      '''
      enc_inputs: [batch_size, src_len]
      dec_inputs: [batch_size, tgt_len]
      dec_outputs: [batch_size, tgt_len]
      '''
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.cuda(), dec_inputs.cuda(), dec_outputs.cuda()
      # outputs: [batch_size * tgt_len, tgt_vocab_size]
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      loss = criterion(outputs, dec_outputs.view(-1))
      print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch: 0001 loss = 2.467164
Epoch: 0002 loss = 2.318526
Epoch: 0003 loss = 2.207512
Epoch: 0004 loss = 1.934394
Epoch: 0005 loss = 1.690412
Epoch: 0006 loss = 1.547174
Epoch: 0007 loss = 1.312358
Epoch: 0008 loss = 1.063020
Epoch: 0009 loss = 0.782508
Epoch: 0010 loss = 0.601187
Epoch: 0011 loss = 0.480187
Epoch: 0012 loss = 0.363662
Epoch: 0013 loss = 0.300485
Epoch: 0014 loss = 0.228560
Epoch: 0015 loss = 0.151120
Epoch: 0016 loss = 0.152684
Epoch: 0017 loss = 0.108565
Epoch: 0018 loss = 0.085597
Epoch: 0019 loss = 0.075393
Epoch: 0020 loss = 0.064178


## 对训练出的模型测试
- 在测试中我们是全部原语言编码后放入model.encoder()得到enc_outputs。根据这个enc_outputs和起始单词"S"作为dec_input，放入到model.decoder()中，则会得到预测结果“i”（由于是贪心算法所以只输出概率最大的那个字符）。
- 接着我们合并“S”和“i”得到新的“S i”作为新一轮预测的dec_input，并和之前得到的enc_outputs再次放入到model.decoder()中，则会得到第二轮的预测结果“want”
- 接着继续合并"S i want"作为新的dec_input。。。。。
- 最后当预测姐结果是“E“时，退出。输出最终的结果

参数解释：
- model: Transformer Model
- enc_input: 编码器输入
- start_symbol: 开始字符，本例中"S"坐标为"7"

函数解释：
- torch.zeros(a,b) : 返回一个形状为为size,类型为torch.dtype，里面的每一个值都是0的tensor
- tensor.data: 生成的都是无梯度的纯tensor
- tensor.type_as(tensor1): 把tensor的类型转化为tensor1的类型
- torch.cat((A,B), dim): 把tensorA和tensorB，根据dim的维度拼接
- tensor.max(dim, keepdim = False): 对tensor根据哪一个维度来求最大值，返回的是‘元组'第0元素表示最大的值为多少，第1元素表示所在位置(类似tensor.argmax(dim))。如果keepdim = False返回的数值维度会在原来的维度上减一

In [82]:
def greedy_decoder(model, enc_input, start_symbol):
    '''
    使用贪心算法：即提取可能性最大的单词作为结果去预测下一个
    enc_input:[batch_size, src_len]
    start_symbol: 开始符号index
    '''
    enc_outputs, enc_self_attns = model.encoder(enc_input)  # enc_outputs：[batch_size, src_len, d_model]
    dec_input = torch.zeros(1, 0).type_as(enc_input.data)   # tensor([], size=(1, 0))
    terminal = False    # 是否终结预测
    next_symbol = start_symbol

    while not terminal:       
        # 通过不断整合上一个预测出的单词，最后在预测到E字符时停止
        dec_input = torch.cat([dec_input.detach().cpu(), torch.tensor([[next_symbol]], dtype=enc_input.dtype)], -1) # [1,1]—>[1,2]->[1,3]...
        dec_outputs, _, _ = model.decoder(dec_input.cuda(), enc_input, enc_outputs)    # [batch_size, tgt_len, d_model]
        projected = model.projection(dec_outputs)      # [batch_size, tgt_len, tgt_vocab_size]
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]   # 得到最大值所在index
        next_word = prob.data[-1] # 取出下一个单词，因为在迭代过程中prob长度不断增大
        next_symbol = next_word
        if next_symbol == tgt_vocab["E"]:
            terminal = True
                    
    return dec_input  # [1,tgt_len]

# Test
enc_inputs, _, _ = next(iter(train_loader))
enc_inputs = enc_inputs.cuda()
predict = greedy_decoder(model, enc_inputs[1].view(1, -1), start_symbol=tgt_vocab["S"])
print(enc_inputs[1], '->', [idx2word[n.item()] for n in predict.squeeze()])

tensor([1, 2, 3, 5, 0], device='cuda:0') -> ['S', 'i', 'want', 'a', 'coke', '.']
